In [1]:
import requests
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib import style
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

%matplotlib inline
style.use('seaborn')

In [2]:
nba_teams = pd.DataFrame(teams.get_teams())
team_ids = nba_teams['id'].unique()

In [189]:
# main_df.groupby(main_df.SEASON_ID.str[-4:])[['GAME_ID']].count()
#2014 season dates
start_2014 = '2014-10-28'
end_2014 = '2015-04-15'

#2015 season dates
start_2015 = '2015-10-27'
end_2015 = '2016-04-13'

#2016 season dates
start_2016 = '2016-10-25'
end_2016 = '2017-04-12'

#2017 season dates
start_2017 = '2017-10-17'
end_2017 = '2018-04-11'

#2018 season dates
start_2018 = '2018-10-16'
end_2018 = '2019-04-10'

#2019 season dates
start_2019 = '2019-10-22'
end_2019 = '2020-03-11' #ends earlier cos after that we head into the bubble

#2020 season dates
start_2020 = '2020-12-22'

#Repulling the data and computing everything
adjusted_main_df = pd.DataFrame()
for team_id in team_ids:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    games = gamefinder.get_data_frames()[0]
    games = games[(games['GAME_DATE'].between(start_2014,end_2014)) | (games['GAME_DATE'].between(start_2015,end_2015)) | (games['GAME_DATE'].between(start_2016,end_2016)) | (games['GAME_DATE'].between(start_2017,end_2017)) | (games['GAME_DATE'].between(start_2018,end_2018)) | (games['GAME_DATE'].between(start_2019,end_2019)) | (games['GAME_DATE']>start_2020)]
    adjusted_main_df = adjusted_main_df.append(games)
    time.sleep(5)

adjusted_main_df = adjusted_main_df.sort_values('GAME_DATE',ascending=True)
adjusted_main_df['EFG'] = ( adjusted_main_df['FGM'] + 0.5 * adjusted_main_df['FG3M'] ) / adjusted_main_df['FGA']
adjusted_main_df.tail()



,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,EFG
0,22020,1610612754,IND,Indiana Pacers,0022000258,2021-01-25,IND vs. TOR,W,240,129,...,7.0,38.0,45.0,26,5.0,8,13,25,10.8,0.589744
0,22020,1610612753,ORL,Orlando Magic,0022000259,2021-01-25,ORL vs. CHA,W,240,117,...,10.0,39.0,49.0,31,10.0,3,15,21,9.6,0.585227
0,22020,1610612751,BKN,Brooklyn Nets,0022000260,2021-01-25,BKN vs. MIA,None,118,38,...,3.0,15.0,18.0,11,2.0,2,3,5,-3.4,0.362500
0,22020,1610612765,DET,Detroit Pistons,0022000257,2021-01-25,DET vs. PHI,W,241,119,...,11.0,36.0,47.0,22,6.0,6,18,20,15.0,0.621429
0,22020,1610612741,CHI,Chicago Bulls,0022000262,2021-01-25,CHI vs. BOS,None,122,46,...,3.0,19.0,22.0,13,2.0,6,8,7,-9.6,0.523810


In [190]:
master_data = adjusted_main_df

Create a team dictionary to store all the different statistics for all teams

In [208]:
nba_teams = {
    'ATL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'BKN': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'BOS': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'CHA': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'CHI': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'CLE': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'DAL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'DEN': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'DET': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'GSW': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'HOU': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'IND': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'LAC': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'LAL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MEM': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MIA': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MIL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'MIN': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'NOP': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'NYK': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'OKC': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'ORL': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'PHI': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'PHX': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'POR': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'SAC': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'SAS': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'TOR': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'UTA': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB': 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 },
    'WAS': {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
            'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
           'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DREB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0,
           "cFTA":0 }
    
}

Determining the columns 

In [209]:
master_data= master_data.sort_values(by = "GAME_DATE", ascending = True)
master_data.index = range(len(master_data))
master_data.head()
master_data.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'EFG',
       'HOME_COURT', 'BEF_AVG_PTS', 'BEF_AVG_AST', 'BEF_AVG_OREB',
       'BEF_AVG_DREB', 'AFT_AVG_PTS', 'AFT_AVG_AST', 'AFT_AVG_OREB',
       'AFT_AVG_DREB', 'GAME_PLAYED', 'cPTS', 'cAST', 'cOREB', 'cDREB', 'cFGA',
       'cTO', 'cFTA', 'DIS_PTS', 'DIS_AST', 'DIS_OREB', 'DIS_DREB', 'OFFRATE',
       'DEFRATE'],
      dtype='object')

combining the opponent team and home team data set onto the same row

In [210]:
master_data['HOME_COURT'] = np.where(master_data['MATCHUP'].str.contains('vs.'), 'H', 'A')
master_data['BEF_AVG_PTS'] = 0
master_data['BEF_AVG_AST'] = 0
master_data['BEF_AVG_OREB'] = 0
master_data['BEF_AVG_DREB'] = 0
master_data['AFT_AVG_PTS'] = 0
master_data['AFT_AVG_AST'] = 0
master_data['AFT_AVG_OREB'] = 0
master_data['AFT_AVG_DREB'] = 0
master_data['GAME_PLAYED'] = 0
master_data['cPTS'] = 0
master_data['cAST'] = 0
master_data['cOREB'] = 0
master_data['cDREB'] = 0
master_data['cFGA'] = 0
master_data['cTO'] = 0
master_data['cFTA'] = 0

master_data['DIS_PTS'] = 0
master_data['DIS_AST'] = 0
master_data['DIS_OREB'] = 0
master_data['DIS_DREB'] = 0

master_data["OFFRATE"] = 0
master_data['DEFRATE'] = 0


master_data.head()
master_data_combined = master_data.merge(master_data, on='GAME_ID')
master_data_combined = master_data_combined.drop(master_data_combined[master_data_combined['TEAM_ID_x'] == master_data_combined['TEAM_ID_y']].index)
master_data_combined = master_data_combined.iloc[::2]

master_data_combined.tail()
# master_data_combined['GAME_PLAYED_x']

,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,cDREB_y,cFGA_y,cTO_y,cFTA_y,DIS_PTS_y,DIS_AST_y,DIS_OREB_y,DIS_DREB_y,OFFRATE_y,DEFRATE_y
29433,22020,1610612738,BOS,Boston Celtics,0022000262,2021-01-25,BOS @ CHI,None,86,44,...,0,0,0,0,0,0,0,0,0,0
29437,22020,1610612761,TOR,Toronto Raptors,0022000258,2021-01-25,TOR @ IND,L,241,114,...,0,0,0,0,0,0,0,0,0,0
29441,22020,1610612739,CLE,Cleveland Cavaliers,0022000261,2021-01-25,CLE vs. LAL,None,181,89,...,0,0,0,0,0,0,0,0,0,0
29445,22020,1610612751,BKN,Brooklyn Nets,0022000260,2021-01-25,BKN vs. MIA,None,118,38,...,0,0,0,0,0,0,0,0,0,0
29449,22020,1610612743,DEN,Denver Nuggets,0022000265,2021-01-25,DEN @ DAL,None,120,76,...,0,0,0,0,0,0,0,0,0,0


Separate into different seasons 

In [211]:
season_2014 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22014']
season_2014.index = range(len(season_2014))

season_2015 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22015']
season_2015.index = range(len(season_2015))

season_2016 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22016']
season_2016.index = range(len(season_2016))

season_2017 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22017']
season_2017.index = range(len(season_2017))


season_2018 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22018']
season_2018.index = range(len(season_2018))

season_2019 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22019']
season_2019.index = range(len(season_2019))

season_2020 = master_data_combined[master_data_combined['SEASON_ID_x'] == '22020']
season_2020.index = range(len(season_2020))

season_2014.head()

# {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
#             'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
#            'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0
#            "cFTA":0 }

,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,cDREB_y,cFGA_y,cTO_y,cFTA_y,DIS_PTS_y,DIS_AST_y,DIS_OREB_y,DIS_DREB_y,OFFRATE_y,DEFRATE_y
0,22014,1610612745,HOU,Houston Rockets,0021400003,2014-10-28,HOU @ LAL,W,241,108,...,0,0,0,0,0,0,0,0,0,0
1,22014,1610612759,SAS,San Antonio Spurs,0021400002,2014-10-28,SAS vs. DAL,W,241,101,...,0,0,0,0,0,0,0,0,0,0
2,22014,1610612740,NOP,New Orleans Pelicans,0021400001,2014-10-28,NOP vs. ORL,W,240,101,...,0,0,0,0,0,0,0,0,0,0
3,22014,1610612741,CHI,Chicago Bulls,0021400010,2014-10-29,CHI @ NYK,W,240,104,...,0,0,0,0,0,0,0,0,0,0
4,22014,1610612737,ATL,Atlanta Hawks,0021400008,2014-10-29,ATL @ TOR,L,240,102,...,0,0,0,0,0,0,0,0,0,0


In [201]:
season_2014_test = season_2014.loc[:100]

season_2014_test= season_2014_test.sort_values(by = "GAME_DATE_x", ascending = True)
season_2014_test.index = range(len(season_2014_test))
season_2014_test

,SEASON_ID_x,TEAM_ID_x,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE_x,MATCHUP_x,WL_x,MIN_x,PTS_x,...,cDREB_y,cFGA_y,cTO_y,cFTA_y,DIS_PTS_y,DIS_AST_y,DIS_OREB_y,DIS_DREB_y,OFFRATE_y,DEFRATE_y
0,22014,1610612745,HOU,Houston Rockets,0021400003,2014-10-28,HOU @ LAL,W,241,108,...,0,0,0,0,0,0,0,0,0,0
1,22014,1610612759,SAS,San Antonio Spurs,0021400002,2014-10-28,SAS vs. DAL,W,241,101,...,0,0,0,0,0,0,0,0,0,0
2,22014,1610612740,NOP,New Orleans Pelicans,0021400001,2014-10-28,NOP vs. ORL,W,240,101,...,0,0,0,0,0,0,0,0,0,0
3,22014,1610612745,HOU,Houston Rockets,0021400012,2014-10-29,HOU @ UTA,W,240,104,...,0,0,0,0,0,0,0,0,0,0
4,22014,1610612765,DET,Detroit Pistons,0021400011,2014-10-29,DET @ DEN,L,241,79,...,0,0,0,0,0,0,0,0,0,0
5,22014,1610612749,MIL,Milwaukee Bucks,0021400004,2014-10-29,MIL @ CHA,L,264,106,...,0,0,0,0,0,0,0,0,0,0
6,22014,1610612755,PHI,Philadelphia 76ers,0021400005,2014-10-29,PHI @ IND,L,240,91,...,0,0,0,0,0,0,0,0,0,0
7,22014,1610612758,SAC,Sacramento Kings,0021400014,2014-10-29,SAC vs. GSW,L,240,77,...,0,0,0,0,0,0,0,0,0,0
8,22014,1610612757,POR,Portland Trail Blazers,0021400015,2014-10-29,POR vs. OKC,W,240,106,...,0,0,0,0,0,0,0,0,0,0
9,22014,1610612748,MIA,Miami Heat,0021400007,2014-10-29,MIA vs. WAS,W,239,107,...,0,0,0,0,0,0,0,0,0,0


Function used to append stats into dataset

In [212]:
for i, row in season_2014.iterrows():
#         get the team name
    team_1 = row['TEAM_ABBREVIATION_x']
    team_2 = row['TEAM_ABBREVIATION_y']
    
#     print(team_1)
#     print(team_2)

#     print(nba_teams[team_1]['GAME_PLAYED'] )
    
#     update team 1 and 2
    if(nba_teams[team_1]['GAME_PLAYED'] == 0):
# # #         team 1
        nba_teams[team_1]['GAME_PLAYED'] = 1
# #         print(nba_teams[team_1]['GAME_PLAYED'])
# #         print("------")
        print("Team"+ team_1+ "Game Played is "+ str(nba_teams[team_1]['GAME_PLAYED']))
        season_2014.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
    
        nba_teams[team_1]['cPTS'] = row['PTS_x']
        nba_teams[team_1]['cAST'] = row['AST_x']
        nba_teams[team_1]['cOREB'] = row['OREB_x']
        nba_teams[team_1]['cDREB'] = row['DREB_x']
        nba_teams[team_1]['cFGA'] = row['FGA_x']
        nba_teams[team_1]['cTO'] = row['TOV_x']
        nba_teams[team_1]['cFTA'] = row['FTA_x']
        

        nba_teams[team_1]['AFT_AVG_PTS'] = nba_teams[team_1]['cPTS']
        nba_teams[team_1]['AFT_AVG_AST'] = nba_teams[team_1]['cAST']
        nba_teams[team_1]['AFT_AVG_OREB'] = nba_teams[team_1]['cOREB']
        nba_teams[team_1]['AFT_AVG_DREB'] = nba_teams[team_1]['cDREB']
        
#                 calculating off def ratings for team 1 
        tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
        off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
#         nba_teams[team_1]['OFFRATE'] = off_ratings_1
        
        def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
        nba_teams[team_1]['DEFRATE'] = def_ratings_1
        
# # #         DISPARITY BETWEEN 2 TEAMS 
        nba_teams[team_1]['DIS_PTS'] = 0 
        nba_teams[team_1]['DIS_AST'] = 0
        nba_teams[team_1]['DIS_OREB'] = 0
        nba_teams[team_1]['DIS_DREB'] = 0
        
    elif(nba_teams[team_1]['GAME_PLAYED'] != 0):
# #         add a before avg ratings for the previous gams 
        nba_teams[team_1]['BEF_AVG_PTS'] = nba_teams[team_1]['AFT_AVG_PTS']
        nba_teams[team_1]['BEF_AVG_AST'] = nba_teams[team_1]['AFT_AVG_AST']
        nba_teams[team_1]['BEF_AVG_OREB'] = nba_teams[team_1]['AFT_AVG_OREB']
        nba_teams[team_1]['BEF_AVG_DREB'] = nba_teams[team_1]['AFT_AVG_DREB']
#         print('-----')
        nba_teams[team_1]['GAME_PLAYED'] = nba_teams[team_1]["GAME_PLAYED"] + 1
        season_2014.loc[i,'GAME_PLAYED_x'] = nba_teams[team_1]['GAME_PLAYED']
        print("Team"+ team_1+ "Game Played is " +str(nba_teams[team_1]['GAME_PLAYED']))
    
        nba_teams[team_1]['cPTS'] += row['PTS_x']
        nba_teams[team_1]['cAST'] += row['AST_x']
        nba_teams[team_1]['cOREB'] += row['OREB_x']
        nba_teams[team_1]['cDREB'] += row['DREB_x']
        nba_teams[team_1]['cFGA'] += row['FGA_x']
        nba_teams[team_1]['cTO'] += row['TOV_x']
        nba_teams[team_1]['cFTA'] += row['FTA_x']
        
        nba_teams[team_1]['AFT_AVG_PTS'] = nba_teams[team_1]['cPTS'] /nba_teams[team_1]["GAME_PLAYED"]
        nba_teams[team_1]['AFT_AVG_AST'] = nba_teams[team_1]['cAST']/nba_teams[team_1]["GAME_PLAYED"]
        nba_teams[team_1]['AFT_AVG_OREB'] = nba_teams[team_1]['cOREB']/nba_teams[team_1]["GAME_PLAYED"]
        nba_teams[team_1]['AFT_AVG_DREB'] = nba_teams[team_1]['cDREB']/nba_teams[team_1]["GAME_PLAYED"]

        
# #                  calculating off def ratings for team 1 
        tot_pos_1 = nba_teams[team_1]['cFGA'] - nba_teams[team_1]['cOREB'] + nba_teams[team_1]['cTO'] +(0.4* nba_teams[team_1]['cFTA'])
        off_ratings_1 = nba_teams[team_1]['cPTS']/tot_pos_1
        nba_teams[team_1]['OFFRATE'] = off_ratings_1
        
        def_ratings_1 = nba_teams[team_2]['cPTS']/tot_pos_1
        nba_teams[team_1]['DEFRATE'] = def_ratings_1
# #                 DISPARITY BETWEEN 2 TEAMS 
        nba_teams[team_1]['DIS_PTS'] = nba_teams[team_1]['BEF_AVG_PTS'] - nba_teams[team_2]['BEF_AVG_PTS']
        nba_teams[team_1]['DIS_AST'] = nba_teams[team_1]['BEF_AVG_AST'] - nba_teams[team_2]['BEF_AVG_AST']
        nba_teams[team_1]['DIS_OREB'] = nba_teams[team_1]['BEF_AVG_OREB'] - nba_teams[team_2]['BEF_AVG_OREB']
        nba_teams[team_1]['DIS_DREB'] = nba_teams[team_1]['BEF_AVG_DREB'] - nba_teams[team_2]['BEF_AVG_DREB']
        
    if(nba_teams[team_2]['GAME_PLAYED'] == 0):
        
# #         team 2 
        nba_teams[team_2]['GAME_PLAYED'] = 1
        season_2014.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED'] 
        print("Team"+ team_2+ "Game Played is " +str(nba_teams[team_2]['GAME_PLAYED']))
#         print(nba_teams[team_2]['GAME_PLAYED'])
#         print("------")
        nba_teams[team_2]['cPTS'] += row['PTS_y']
        nba_teams[team_2]['cAST'] += row['AST_y']
        nba_teams[team_2]['cOREB'] += row['OREB_y']
        nba_teams[team_2]['cDREB'] += row['DREB_y']
        nba_teams[team_2]['cFGA'] += row['FGA_y']
        nba_teams[team_2]['cTO'] += row['TOV_y']
        nba_teams[team_2]['cFTA'] += row['FTA_y']
        

        nba_teams[team_2]['AFT_AVG_PTS'] = nba_teams[team_2]['cPTS']
        nba_teams[team_2]['AFT_AVG_AST'] = nba_teams[team_2]['cAST']
        nba_teams[team_2]['AFT_AVG_OREB'] = nba_teams[team_2]['cOREB']
        nba_teams[team_2]['AFT_AVG_DREB'] = nba_teams[team_2]['cDREB']
        
        
# #         #         calculating off def ratings for team 2
        tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
        off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
        nba_teams[team_2]['OFFRATE'] = off_ratings_2
        
        def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
        nba_teams[team_2]['DEFRATE'] = def_ratings_2
        
# #         #         DISPARITY BETWEEN 2 TEAMS 
        nba_teams[team_2]['DIS_PTS'] = 0
        nba_teams[team_2]['DIS_AST'] = 0
        nba_teams[team_2]['DIS_OREB'] = 0
        nba_teams[team_2]['DIS_DREB'] = 0
    

        
        
    elif(nba_teams[team_2]['GAME_PLAYED'] != 0):
# # #         add a before avg ratings for the previous gams 
        nba_teams[team_2]['BEF_AVG_PTS'] = nba_teams[team_2]['AFT_AVG_PTS']
        nba_teams[team_2]['BEF_AVG_AST'] = nba_teams[team_2]['AFT_AVG_AST']
        nba_teams[team_2]['BEF_AVG_OREB'] = nba_teams[team_2]['AFT_AVG_OREB']
        nba_teams[team_2]['BEF_AVG_DREB'] = nba_teams[team_2]['AFT_AVG_DREB']
        
    
        nba_teams[team_2]['GAME_PLAYED'] = nba_teams[team_2]["GAME_PLAYED"] + 1
        
        season_2014.loc[i,'GAME_PLAYED_y'] = nba_teams[team_2]['GAME_PLAYED']   
        print("Team"+ team_2+ "Game Played is "+ str(nba_teams[team_2]['GAME_PLAYED']))
        
#         print(nba_teams[team_2]['GAME_PLAYED'])
#         print('hi')
#         print("------")
        nba_teams[team_2]['cPTS'] += row['PTS_y']
        nba_teams[team_2]['cAST'] += row['AST_y']
        nba_teams[team_2]['cOREB'] += row['OREB_y']
        nba_teams[team_2]['cDREB'] += row['DREB_y']
        nba_teams[team_2]['cFGA'] += row['FGA_y']
        nba_teams[team_2]['cTO'] += row['TOV_y']
        nba_teams[team_2]['cFTA'] += row['FTA_y']

        nba_teams[team_2]['AFT_AVG_PTS'] = nba_teams[team_2]['cPTS'] /nba_teams[team_2]["GAME_PLAYED"]
        nba_teams[team_2]['AFT_AVG_AST'] = nba_teams[team_2]['cAST']/nba_teams[team_2]["GAME_PLAYED"]
        nba_teams[team_2]['AFT_AVG_OREB'] = nba_teams[team_2]['cOREB']/nba_teams[team_2]["GAME_PLAYED"]
        nba_teams[team_2]['AFT_AVG_DREB'] = nba_teams[team_2]['cDREB']/nba_teams[team_2]["GAME_PLAYED"]
        
# #          #         calculating off def ratings for team 2
        tot_pos_2 = nba_teams[team_2]['cFGA'] - nba_teams[team_2]['cOREB'] + nba_teams[team_2]['cTO'] +(0.4* nba_teams[team_2]['cFTA'])
        off_ratings_2 = nba_teams[team_2]['cPTS']/tot_pos_2
        nba_teams[team_2]['OFFRATE'] = off_ratings_2
        
        def_ratings_2 = nba_teams[team_1]['cPTS']/tot_pos_2
        nba_teams[team_2]['DEFRATE'] = def_ratings_2
#                 #         DISPARITY BETWEEN 2 TEAMS 
        nba_teams[team_2]['DIS_PTS'] = nba_teams[team_2]['BEF_AVG_PTS'] - nba_teams[team_1]['BEF_AVG_PTS']
        nba_teams[team_2]['DIS_AST'] = nba_teams[team_2]['BEF_AVG_AST'] - nba_teams[team_1]['BEF_AVG_AST']
        nba_teams[team_2]['DIS_OREB'] = nba_teams[team_2]['BEF_AVG_OREB'] - nba_teams[team_1]['BEF_AVG_OREB']
        nba_teams[team_2]['DIS_DREB'] = nba_teams[team_2]['BEF_AVG_DREB'] - nba_teams[team_1]['BEF_AVG_DREB']
    
# #     add in the data into the dataset 
# #         add_into_dataset(season_2014)
    season_2014.loc[i,'BEF_AVG_PTS_x'] = nba_teams[team_1]['BEF_AVG_PTS']
    season_2014.loc[i,'BEF_AVG_PTS_y'] = nba_teams[team_2]['BEF_AVG_PTS']
    season_2014.loc[i,'BEF_AVG_AST_x'] = nba_teams[team_1]['BEF_AVG_AST']
    season_2014.loc[i,'BEF_AVG_AST_y'] = nba_teams[team_2]['BEF_AVG_AST']
    season_2014.loc[i,'BEF_AVG_OREB_x'] = nba_teams[team_1]['BEF_AVG_OREB']
    season_2014.loc[i,'BEF_AVG_OREB_y'] = nba_teams[team_2]['BEF_AVG_OREB']
    season_2014.loc[i,'BEF_AVG_DREB_x'] = nba_teams[team_1]['BEF_AVG_DREB']
    season_2014.loc[i,'BEF_AVG_DREB_y'] = nba_teams[team_2]['BEF_AVG_DREB']
    
    season_2014.loc[i,'AFT_AVG_PTS_x'] = nba_teams[team_1]['AFT_AVG_PTS']
    season_2014.loc[i,'AFT_AVG_PTS_y'] = nba_teams[team_2]['AFT_AVG_PTS']
    season_2014.loc[i,'AFT_AVG_AST_x'] = nba_teams[team_1]['AFT_AVG_AST']
    season_2014.loc[i,'AFT_AVG_AST_y'] = nba_teams[team_2]['AFT_AVG_AST']
    season_2014.loc[i,'AFT_AVG_OREB_x'] = nba_teams[team_1]['AFT_AVG_OREB']
    season_2014.loc[i,'AFT_AVG_OREB_y'] = nba_teams[team_2]['AFT_AVG_OREB']
    season_2014.loc[i,'AFT_AVG_DREB_x'] = nba_teams[team_1]['AFT_AVG_DREB']
    season_2014.loc[i,'AFT_AVG_DREB_y'] = nba_teams[team_2]['AFT_AVG_DREB']
    
                                                   
    season_2014.loc[i,'cPTS_x'] = nba_teams[team_1]['cPTS']
    season_2014.loc[i,'cAST_x'] = nba_teams[team_1]['cAST']
    season_2014.loc[i,'cOREB_x'] = nba_teams[team_1]['cOREB']
    season_2014.loc[i,'cDREB_x'] = nba_teams[team_1]['cDREB']
    season_2014.loc[i,'cFGA_x'] = nba_teams[team_1]['cFGA']
    season_2014.loc[i,'cTO_x'] = nba_teams[team_1]['cTO']
    season_2014.loc[i,'cFTA_x'] = nba_teams[team_1]['cFTA']
    season_2014.loc[i,'cPTS_y'] = nba_teams[team_2]['cPTS']
    season_2014.loc[i,'cAST_y'] = nba_teams[team_2]['cAST']
    season_2014.loc[i,'cOREB_y'] = nba_teams[team_2]['cOREB']
    season_2014.loc[i,'cDREB_y'] = nba_teams[team_2]['cDREB']
    season_2014.loc[i,'cFGA_y'] = nba_teams[team_2]['cFGA']
    season_2014.loc[i,'cTO_y'] = nba_teams[team_2]['cTO']
    season_2014.loc[i,'cFTA_y'] = nba_teams[team_2]['cFTA']

    season_2014.loc[i,'DIS_PTS_x'] = nba_teams[team_1]['DIS_PTS']
    season_2014.loc[i,'DIS_AST_x'] = nba_teams[team_1]['DIS_AST']
    season_2014.loc[i,'DIS_OREB_x'] = nba_teams[team_1]['DIS_OREB']
    season_2014.loc[i,'DIS_DREB_x'] = nba_teams[team_1]['DIS_DREB']
    season_2014.loc[i,'DIS_PTS_y'] = nba_teams[team_2]['DIS_PTS']
    season_2014.loc[i,'DIS_AST_y'] = nba_teams[team_2]['DIS_AST']
    season_2014.loc[i,'DIS_OREB_y'] = nba_teams[team_2]['DIS_OREB']
    season_2014.loc[i,'DIS_DREB_y'] = nba_teams[team_2]['DIS_DREB']

    season_2014.loc[i,"OFFRATE_x"] = nba_teams[team_1]['OFFRATE']
    season_2014.loc[i,'DEFRATE_x'] = nba_teams[team_1]['DEFRATE']
    season_2014.loc[i,"OFFRATE_y"] = nba_teams[team_2]['OFFRATE']
    season_2014.loc[i,'DEFRATE_y'] = nba_teams[team_2]['DEFRATE']                                                            
    
        
        

TeamHOUGame Played is 1
TeamLALGame Played is 1
TeamSASGame Played is 1
TeamDALGame Played is 1
TeamNOPGame Played is 1
TeamORLGame Played is 1
TeamCHIGame Played is 1
TeamNYKGame Played is 1
TeamATLGame Played is 1
TeamTORGame Played is 1
TeamMIAGame Played is 1
TeamWASGame Played is 1
TeamBKNGame Played is 1
TeamBOSGame Played is 1
TeamPHIGame Played is 1
TeamINDGame Played is 1
TeamPORGame Played is 1
TeamOKCGame Played is 1
TeamMILGame Played is 1
TeamCHAGame Played is 1
TeamPHXGame Played is 1
TeamLALGame Played is 2
TeamSACGame Played is 1
TeamGSWGame Played is 1
TeamDETGame Played is 1
TeamDENGame Played is 1
TeamMINGame Played is 1
TeamMEMGame Played is 1
TeamHOUGame Played is 2
TeamUTAGame Played is 1
TeamCLEGame Played is 1
TeamNYKGame Played is 2
TeamMINGame Played is 2
TeamDETGame Played is 2
TeamDALGame Played is 2
TeamUTAGame Played is 2
TeamLACGame Played is 1
TeamOKCGame Played is 2
TeamORLGame Played is 2
TeamWASGame Played is 2
TeamLACGame Played is 2
TeamLALGame Play

In [213]:
season_2014.to_csv('2014_nba_data.csv',index=False) 

code test

# {'GAME_PLAYED': 0, 'BEF_AVG_PTS':0,'AFT_AVG_PTS':0,'cPTS':0,'cAST':0,'cOREB':0,'cDREB':0,
#             'BEF_AVG_AST':0,'BEF_AVG_OREB':0, 'BEF_AVG_DREB':0,'AFT_AVG_AST':0,'AFT_AVG_OREB':0, 'AFT_AVG_DREB':0,
#            'DIS_PTS' : 0,'DIS_AST' : 0, 'DIS_OREB' : 0,'DIS_DB' : 0, 'OFFRATE':0, 'DEFRATE':0, "cFGA":0, "cTO": 0
#            "cFTA":0 }